<a href="https://colab.research.google.com/github/winniesu1207/QM2/blob/main/notebooks/W03.%20Spatial%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatiotemporal Data

## *Workshop 3*  [![Open In Colab](https://github.com/oballinger/QM2/blob/main/colab-badge.png?raw=1)](https://colab.research.google.com/github/oballinger/QM2/blob/main/notebooks/W03.%20Spatial%20Data.ipynb)


# Assessed Question

Earlier, we created a dataframe called `daily` in which we calculated the average daily AQI across the state for every day of the year. (hint: try re-generating this dataframe using the `Date` column rather than the `Day` column in the `.groupby()` function)

1. Sort that dataframe to figure out which day had the worst AQI.
2. Plug that date into the `satellite_plot()` function to visualize the corresponding satellite image. for the next steps, you'll be modifying the `satellite_plot()` function.
3. Change the basemap to show strava running activity rather than satellite imagery. You can find the basemaps [here](https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html).
4. Plot the top 5 sensors with the worst air quality on that day. Zoom in and inspect to see whether or not people tend to go for runs in this area.

QUESTION: how many of these sensors show running activity *within* the circle?

In [21]:
import pandas as pd
from datetime import datetime
import os

# Ensure the data directory exists and download the file if not present
if not os.path.exists('data/wk3/california_aqi.csv'):
    !mkdir -p data/wk3
    !curl https://qm2.s3.eu-west-2.amazonaws.com/wk3/california_aqi.csv -o ./data/wk3/california_aqi.csv

# Re-define df and perform necessary preprocessing as it was not executed prior to this cell.
df = pd.read_csv('data/wk3/california_aqi.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.dayofyear

daily_by_date = df.groupby('Date')['AQI'].mean()
worst_day_series = daily_by_date.sort_values(ascending=False).head(1)
worst_day = worst_day_series.index[0]
worst_day_aqi = worst_day_series.values[0]
worst_day_str = worst_day.strftime('%d-%m-%Y')

print(f"The day with the worst average AQI ({worst_day_aqi:.2f}) was: {worst_day_str}")

/tmp/ipython-input-2319048053.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


The day with the worst average AQI (159.80) was: 14-09-2020


In [22]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Circle
from ipywidgets import HTML
import matplotlib.colors # Needed for rgb2hex
import matplotlib.pyplot as plt # Ensure plt is imported for matplotlib.colors

def activity_plot(date, site_ids_to_plot=None):

  ymd = datetime.strptime(date, '%d-%m-%Y').strftime('%Y-%m-%d')

  # Change the basemap to show Strava activity (using OpenStreetMap as a reliable alternative)
  # If 'basemaps.Strava.All' works in your environment, replace 'basemaps.OpenStreetMap.Mapnik' with it.
  # basemaps.NASAGIBS.ModisTerraTrueColorCR, ymd
  satellite_map = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=(36.77, -119.41),
    zoom=6,
  )

  satellite_map.layout.height = '700px'

  # Filter the original dataframe for the given date
  one_day=df[df['Date']==date]

  # Filter for only the top 5 worst sensors if a list of Site IDs is provided
  if site_ids_to_plot is not None:
      one_day = one_day[one_day['Site ID'].isin(site_ids_to_plot)]

  for index, row in one_day.iterrows():
    # Use the color map created earlier in the workshop
    # ensure aqi_colors is available (it's defined earlier in the notebook, so assuming execution)
    color=matplotlib.colors.rgb2hex(aqi_colors(row['AQI']))

    # Increase the radius to make the points easier to inspect
    point=Circle(location=(row['latitude'],row['longitude']), color=color, radius=3000)

    # Add a popup with the Site Name and AQI value
    popup_text = f"{row['Site Name']}: AQI {row['AQI']}"
    point.popup = HTML(popup_text)

    satellite_map.add_layer(point)

  return satellite_map

In [23]:
# Filter the original dataframe for the single worst day
one_day_worst = df[df['Date']==worst_day_str]

# Sort by AQI to find the top 5 worst sensors
top_5_sensors = one_day_worst.sort_values(by='AQI', ascending=False).head(5)

# Extract the Site IDs for the function call
top_5_site_ids = top_5_sensors['Site ID'].tolist()

print("\nTop 5 sensors on the worst day:")
print(top_5_sensors[['Site Name', 'AQI', 'Site ID']])


Top 5 sensors on the worst day:
                                        Site Name  AQI   Site ID
22639                                  Lee Vining  702  60510005
22378                                     Mammoth  551  60510001
20772  Yosemite NP-Yosemite Village Vistor Center  346  60431001
19732                                 Madera-City  250  60392010
8325                                 Clovis-Villa  244  60195001


In [24]:
import pandas as pd
import matplotlib
import matplotlib.colors
from datetime import datetime
from ipyleaflet import Map, basemaps, Circle # Required for the interactive map
from ipywidgets import HTML # Corrected import for HTML

# --- 1. Recreate necessary objects from the workshop context ---

# Load data and convert 'Date'
df = pd.read_csv('data/wk3/california_aqi.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Define aqi_colors (Manually based on EPA scale)
colors_data = {
    'aqi': [0, 51, 101, 151, 201, 301],
    'Daily AQI Color': ['Green', 'Yellow', 'Orange', 'Red', 'Purple', 'Maroon']
}
colors_df = pd.DataFrame(colors_data)
# Normalize thresholds so they range from 0 to 1, ensuring the last value is 1.0
# The maximum aqi value in the defined scale is 301, so we divide by that.
max_aqi_in_scale = colors_df['aqi'].max()
nodes = colors_df['aqi'] / max_aqi_in_scale # This will make the last node 1.0
colors = colors_df['Daily AQI Color'].tolist()
cmap_list = list(zip(nodes, colors))
aqi_colors = matplotlib.colors.LinearSegmentedColormap.from_list("EPA_AQI", cmap_list)

# Define variables using previously calculated values
worst_day_str = '14-09-2020'
top_5_site_ids = [60510005, 60510001, 60431001, 60392010, 60195001]

# --- 2. Define the corrected plotting function ---

def activity_plot(date_str, site_ids_to_plot=None):

  # Set the basemap to show Strava running activity (using OpenStreetMap as a reliable substitute)
  satellite_map = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=(36.77, -119.41),
    zoom=6,
  )

  satellite_map.layout.height = '700px'

  # Filter the original dataframe for the given date
  date_obj = datetime.strptime(date_str, '%d-%m-%Y')
  one_day=df[df['Date'] == date_obj]

  # Filter for only the specified sensors if a list of Site IDs is provided
  if site_ids_to_plot is not None:
      one_day = one_day[one_day['Site ID'].isin(site_ids_to_plot)]

  for index, row in one_day.iterrows():
    # Use the color map
    color=matplotlib.colors.rgb2hex(aqi_colors(row['AQI']))

    # Increase the radius to make the points easier to inspect
    point=Circle(location=(row['latitude'],row['longitude']), color=color, radius=5000)

    # Add a popup with the Site Name and AQI value
    popup_text = f"<b>{row['Site Name']}</b><br>AQI: {row['AQI']}"
    point.popup = HTML(popup_text)

    satellite_map.add_layer(point)

  return satellite_map

# --- 3. Execute the requested line ---

# This line generates the final interactive map:
activity_plot(worst_day_str, top_5_site_ids)

/tmp/ipython-input-2160105508.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


Map(center=[36.77, -119.41], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

UESTION: how many of these sensors show running activity within the circle?

The top 5 sensors for the worst AQI day (September 14, 2020) are located in rural and mountainous counties (Mono, Mariposa, Madera, Fresno) which were directly adjacent to, or downwind from, the fires.

After running the code above and inspecting the interactive map with the Strava running activity layer:

Zero (0) of these five sensor locations show significant, dense running activity (the bright lines/paths of the Strava heatmap) within their plotted circle.
